In [1]:
# Import des librairies
import numpy as np
import pandas as pd

# Question 1 :

* Créez un dataframe contenant les informations de population de chaque pays. 
* Calculez le nombre total d’humains sur la planète. Critiquez votre résultat. 
* En cas d’anomalie, analysez et effectuer les corrections nécessaires. 
* Donnez le résultat de votre calcul pour l'année 2013.

**NB:** 2013 est la dernière année disponible dans les [Food Balance Sheets de la FAO](http://www.fao.org/faostat/en/#data/FBS)<br>

### 1.1 - Data wranging, création du dataframe

In [2]:
# Import des données 'fb_population_2013'
pop = pd.read_csv('data/fb_population_2013.csv')
pop.head(1)

,Domain Code,Domain,Country Code,Country,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,FBS,Food Balance Sheets,2,Afghanistan,511,Total Population - Both sexes,2501,Population,2013,2013,1000 persons,30552,NaN,Official data


In [3]:
# on retire les variables inutiles
pop.drop(['Domain Code','Domain','Element','Element Code','Item Code','Item','Year Code',
          'Year','Unit','Flag'], axis=1, inplace = True)
pop.head(0)

,Country Code,Country,Value,Flag Description


In [4]:
# on multiplie 'population' par son unité = 1000 persons
pop['Value'] = pop['Value'] * 1000

# On renomme les variables de pop
pop.columns = ['ctry_cd','country','population','flag_description']

pop.head(1)

,ctry_cd,country,population,flag_description
0,2,Afghanistan,30552000,Official data


### 1. 2 - Calculez le nombre total d’humains sur la planète. Critiquez votre résultat

In [5]:
# on fixe le résultat
pop_unit = pop.copy()
pop_unit.head(1)

,ctry_cd,country,population,flag_description
0,2,Afghanistan,30552000,Official data


In [6]:
# population mondiale
pop_unit['population'].sum()

8413993000

In [7]:
# nb de pays
pop_unit['ctry_cd'].count()

175

In [8]:
# Valeurs uniques de 'flag_description'
pop_unit['flag_description'].unique()

array(['Official data',
       'Aggregate, may include official, semi-official, estimated or calculated data'],
      dtype=object)

Le résultat est incorrect, la population était de 7,09 milliards en 2013 selon l'[INED](https://www.ined.fr/).  <br>
* Il y a probablement des doublons dans le dataframe <br>
* Or, nous avons un **agrégat** pour la Chine selon la variable 'flag_description'

### 1.3 - En cas d’anomalie, analysez et effectuer les corrections nécessaires

In [9]:
# on sélectionne l'agrégat "China"
china = pop_unit[(pop_unit['country'] == 'China')]
china

,ctry_cd,country,population,flag_description
174,351,China,1416667000,"Aggregate, may include official, semi-official..."


In [10]:
# Recherche des divisions de l'agrégat "China"
china2 = pop_unit[pop_unit['country'].str.contains('China,')]
china2

,ctry_cd,country,population,flag_description
32,96,"China, Hong Kong SAR",7204000,Official data
33,128,"China, Macao SAR",566000,Official data
34,41,"China, mainland",1385567000,Official data
35,214,"China, Taiwan Province of",23330000,Official data


In [11]:
# somme
china2['population'].sum()

1416667000

La population de 'china2' est bien égale à celle de l'agrégat 'china' = 1.416.667.000
* **country China** = Hong Kong SAR + China, Macao SAR + China, mainland + China, Taiwan Province of
* **ctry_cd 351** = 96 + 128 + 41 + 214

### 1.4 - Donnez le résultat de votre calcul pour l'année 2013

In [12]:
pop_clean = pop_unit.copy()
# On exclut l'agrégat "China" :
pop_clean = pop_clean[(pop_clean['country'] != "China")]
# nb de pays
pop_clean['ctry_cd'].count()

174

In [13]:
# population mondiale
pop_clean['population'].sum()

6997326000

**NB:** Le résultat est cohérent, même s'il est d'environ 90 millions en-dessous des chiffres de l'[INED](https://www.ined.fr/) pour 2013
* Mais il y a 174 pays dans les chiffres de la FAO alors que le [nombre officiel de pays comptés à ce jour par l'ONU](https://fr.wikipedia.org/wiki/Liste_des_%C3%89tats_membres_de_l%27Organisation_des_Nations_unies) est de 197
* Par ailleurs, l'écart n'est que de 1,3% avec l'INED
* Et puis après tout, nous travaillons avec les chiffres fournis par la FAO !

### TP1 - union des fichiers 'animal' et 'vegetal' et création d'une variable conjointe

In [14]:
# Import des données des bilans alimentaires
veg = pd.read_csv('data/fb_vegetal_2013.csv')
ani = pd.read_csv('data/fb_animal_2013.csv')

# Ajout de la variable origin
veg['origin'] = 'vegetal'
ani['origin'] = 'animal'

In [15]:
# On fait l'union de 'veg' et de 'ani' dans une nouvelle colonne d'un nouveau df
temp = ani.append(veg)
temp.head(1)

,Domain Code,Domain,Country Code,Country,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description,origin
0,FBS,Food Balance Sheets,2,Afghanistan,5511,Production,2731,Bovine Meat,2013,2013,1000 tonnes,134.0,S,Standardized data,animal


In [16]:
# on multiplie Value par Unit = 1000 pour un résultat en tonnes

In [17]:
# On ne garde que les variables utiles
temp.drop(['Domain Code', 'Domain', 'Element Code', 'Year Code', 'Year', 'Flag', 'Flag Description'], axis=1, inplace = True)
temp.head(1)

,Country Code,Country,Element,Item Code,Item,Unit,Value,origin
0,2,Afghanistan,Production,2731,Bovine Meat,1000 tonnes,134.0,animal


In [18]:
# On renomme les colonnes
temp.columns = ['ctry_cd','country', 'element', 'item_cd', 'item','unit', 'value', 'origin']
temp.head(1)

,ctry_cd,country,element,item_cd,item,unit,value,origin
0,2,Afghanistan,Production,2731,Bovine Meat,1000 tonnes,134.0,animal


### TP2 - fusion des tables 'population' et 'temp' basée sur la clé 'ctry_cd'

In [19]:
# on reprend pop_clean du point 1.4
pop_clean.head(0)

,ctry_cd,country,population,flag_description


* l'agrégat 'China' est exclu pour ne pas fausser d'éventuelles moyennes
* mais nous savons comment le manipuler avec la variable : 'ctry_cd' (**351 =** 96 + 128 + 41 + 214)
* nous faisons une table minimale pour la fusion future

In [20]:
# table minimale
pop_clean.drop(['country', 'flag_description'], axis=1, inplace = True)
pop_clean.head(1)

,ctry_cd,population
0,2,30552000


In [21]:
# fusion de 'temp et de 'pop_clean' sur la clé 'ctry_cd'
fb_clean = pd.merge(temp, pop_clean, on='ctry_cd')
fb_clean.head(2)

,ctry_cd,country,element,item_cd,item,unit,value,origin,population
0,2,Afghanistan,Production,2731,Bovine Meat,1000 tonnes,134.0,animal,30552000
1,2,Afghanistan,Import Quantity,2731,Bovine Meat,1000 tonnes,6.0,animal,30552000


In [22]:
# nb de lignes
fb_clean.value.count()

140919

# Question 2
Parmi les documents sur les Bilans alimentaires que vous avez téléchargés, il y a des informations redondantes. En effet, pour un pays donné, certaines de ces informations peuvent se calculer à partir d’autres :

* Production (Production)
* Importations (Import Quantity)
* Exportations (Export Quantity)
* Variation de stock (Stock Variation)
* Disponibilité intérieure (Domestic supply)
* Semences (Seed)
* Pertes (pas Waste comme le dit lénoncé mais Losses)
* Nourriture (Food) , aussi appelée Disponibilité alimentaire (Food Supply)
* Aliments pour animaux (Feed)
* Traitement (Processing)
* Autres utilisations (Other uses)

Identifiez ces redondances, en donnant votre réponse sous forme de formule mathématique (pas besoin de coder ici :soleil: ). C’est une équation à 3 termes de type **a1+a2+[…]=b1+b2+[…]=c1+c2+[…] )** faisant intervenir chacune des 11 quantités données ci dessus. Illustrez cette équation avec l’exemple du blé en France. Pour avoir un indice, cliquez sur “Définitions et Standards” sur la page de téléchargement des données.

**NB :** Rien sur la page "[définition et standards](http://www.fao.org/faostat/fr/#definitions)" ne nous permet de réaliser l'exercice. en revanche, j'ai trouvé le FAO Handbook datant de 2001. 

**Source:** [Food Balance Sheets Handbook](http://www.fao.org/docrep/003/x9892e/X9892E00.htm#TopOfPage), chapter "[standardization of Food Balance Sheets](http://www.fao.org/docrep/003/x9892e/X9892e04.htm#P3309_98127)"

In [23]:
# on cherche les variables de la colonne 'element'
fb_clean['element'].unique()

array(['Production', 'Import Quantity', 'Domestic supply quantity',
       'Food', 'Food supply quantity (kg/capita/yr)',
       'Food supply (kcal/capita/day)',
       'Protein supply quantity (g/capita/day)',
       'Fat supply quantity (g/capita/day)', 'Seed', 'Losses',
       'Export Quantity', 'Feed', 'Other uses', 'Stock Variation',
       'Processing'], dtype=object)

In [24]:
# exclusion des 4 valeurs qui ne sont pas mentionnées dans l'énoncé
df = fb_clean[~fb_clean['element'].isin(['Food supply quantity (kg/capita/yr)', 'Food supply (kcal/capita/day)',
                                    'Protein supply quantity (g/capita/day)','Fat supply quantity (g/capita/day)'])]
df['element'].unique()

array(['Production', 'Import Quantity', 'Domestic supply quantity',
       'Food', 'Seed', 'Losses', 'Export Quantity', 'Feed', 'Other uses',
       'Stock Variation', 'Processing'], dtype=object)

In [25]:
# prépa du df France = Wheat and products
france=df.copy()
france.loc[(france['country']=="France") & (france['item'] == "Wheat and products"),:]

,ctry_cd,country,element,item_cd,item,unit,value,origin,population
45733,68,France,Production,2511,Wheat and products,1000 tonnes,38614.0,vegetal,64291000
45734,68,France,Import Quantity,2511,Wheat and products,1000 tonnes,2055.0,vegetal,64291000
45735,68,France,Stock Variation,2511,Wheat and products,1000 tonnes,1131.0,vegetal,64291000
45736,68,France,Export Quantity,2511,Wheat and products,1000 tonnes,21502.0,vegetal,64291000
45737,68,France,Domestic supply quantity,2511,Wheat and products,1000 tonnes,20298.0,vegetal,64291000
45738,68,France,Feed,2511,Wheat and products,1000 tonnes,7822.0,vegetal,64291000
45739,68,France,Seed,2511,Wheat and products,1000 tonnes,748.0,vegetal,64291000
45740,68,France,Losses,2511,Wheat and products,1000 tonnes,358.0,vegetal,64291000
45741,68,France,Processing,2511,Wheat and products,1000 tonnes,1575.0,vegetal,64291000
45742,68,France,Other uses,2511,Wheat and products,1000 tonnes,2824.0,vegetal,64291000


#### Je préfère exprimer les redondances des 11 variables sous cette forme :####
**A** = [Feed + Food + Losses + Other uses + Processing + Seed] = [Domestic supply quantity] = **20298**<br>
A = 7822 + 6971 + 358 + 2824 + 1575 + 748 = 20298<br><br>
  **B** = [Export Quantity] = **21502**<br><br>
**A** + **B** = [Domestic supply quantity] + [Export Quantity] = **41800**<br><br>
**C** = [Import Quantity + Production + Stock Variation] = Balance = **41800**<br>
**C** = 20298 + 21502 = 2055 + 38614 + 1131 = 41800<br><br>
**C** = **A** + **B**<br>
Balance = [Domestic supply quantity] + [Export Quantity]

**Source:** [Food Balance Sheets Handbook](http://www.fao.org/docrep/003/x9892e/X9892E00.htm#TopOfPage), chapter "[standardization of Food Balance Sheets](http://www.fao.org/docrep/003/x9892e/X9892e04.htm#P3309_98127)"

### TP3 - pivot de la variable 'element'

In [26]:
# on reprend la table union de population, animal et végétal TP2
fb_clean.head(0)

,ctry_cd,country,element,item_cd,item,unit,value,origin,population


In [27]:
# pivot de la variable 'element'
data = fb_clean.pivot_table(
    index=['ctry_cd','country', 'item_cd','item', 'origin', 'population'],
    columns = ['element'], values=['value'], aggfunc=sum)
data.columns = ['Domestic supply quantity', 'Export Quantity', 'Fat supply quantity (g/capita/day)', 'Feed', 'Food',
                'Food supply (kcal/capita/day)', 'Food supply quantity (kg/capita/yr)','Import Quantity', 'Losses', 
                'Other uses', 'Processing', 'Production', 'Protein supply quantity (g/capita/day)', 'Seed', 'Stock Variation']
data = data.reset_index()
data.head(1)

,ctry_cd,country,item_cd,item,origin,population,Domestic supply quantity,Export Quantity,Fat supply quantity (g/capita/day),Feed,...,Food supply (kcal/capita/day),Food supply quantity (kg/capita/yr),Import Quantity,Losses,Other uses,Processing,Production,Protein supply quantity (g/capita/day),Seed,Stock Variation
0,1,Armenia,2511,Wheat and products,vegetal,2977000,554.0,1.0,3.6,93.0,...,1024.0,130.6,361.0,32.0,0.0,10.0,312.0,30.52,30.0,-118.0


In [28]:
# check
data['origin'].unique()

array(['vegetal', 'animal'], dtype=object)

# Question 3 :

Calculez (pour chaque pays et chaque produit) la disponibilité alimentaire en kcal puis en kg de protéines.<br><br>
**Vous ferez cela à partir de ces informations :**
* Population de chaque pays
* Disponibilité alimentaire ("Food Supply" en anglais) donnée pour chaque produit et pour chaque pays en kcal/personne/jour
* Disponibilité alimentaire en protéines ("Protein supply quantity" en anglais) donnée pour chaque produit et pour chaque pays en g/personne/jour

**NB:** 2013 est la dernière année disponible dans les [Food Balance Sheets de la FAO](http://www.fao.org/faostat/en/#data/FBS)

### 3.1-  solution pour 'Food (kcal/ctry/year)'

In [29]:
# nous repartons du df 'data' de TP3
food = data.copy()
food.head(0)

,ctry_cd,country,item_cd,item,origin,population,Domestic supply quantity,Export Quantity,Fat supply quantity (g/capita/day),Feed,...,Food supply (kcal/capita/day),Food supply quantity (kg/capita/yr),Import Quantity,Losses,Other uses,Processing,Production,Protein supply quantity (g/capita/day),Seed,Stock Variation


In [30]:
# on ne garde que le nécessaire pour répondre à la question 
food.drop(['Production', 'Import Quantity', 'Stock Variation',
       'Domestic supply quantity', 'Seed', 'Losses', 'Food',
       'Fat supply quantity (g/capita/day)', 'Feed', 'Export Quantity',
       'Processing', 'Other uses', 'Food supply quantity (kg/capita/yr)', 'Protein supply quantity (g/capita/day)'], axis=1, inplace = True)
food.head(1)

,ctry_cd,country,item_cd,item,origin,population,Food supply (kcal/capita/day)
0,1,Armenia,2511,Wheat and products,vegetal,2977000,1024.0


In [31]:
# on retire les éventuels NaN
food = food[food['Food supply (kcal/capita/day)'].notnull()]

In [32]:
# création de la colonne 'Food supply (kcal/ctry/y)' = 'Food supply (kcal/capita/day)' x 'population' x 365 jours
food['Food supply (kcal/ctry/y)'] = food['Food supply (kcal/capita/day)'] * food['population'] * 365

# on retire la variable inutile
food.drop(['Food supply (kcal/capita/day)'], axis=1, inplace = True)
food.head(2)

,ctry_cd,country,item_cd,item,origin,population,Food supply (kcal/ctry/y)
0,1,Armenia,2511,Wheat and products,vegetal,2977000,1.112684e+12
1,1,Armenia,2513,Barley and products,vegetal,2977000,0.000000e+00


In [33]:
# Export au format .csv
food.to_csv('data/P3_Q3_food_2013.csv')

### 3.2 - solution pour 'Protein (kg/ctry/year)'

In [34]:
# nous repartons du df 'data' TP3 
protein = data.copy()
protein.head(0)

,ctry_cd,country,item_cd,item,origin,population,Domestic supply quantity,Export Quantity,Fat supply quantity (g/capita/day),Feed,...,Food supply (kcal/capita/day),Food supply quantity (kg/capita/yr),Import Quantity,Losses,Other uses,Processing,Production,Protein supply quantity (g/capita/day),Seed,Stock Variation


In [35]:
# on ne garde que la variable nécessaire pour répondre à la question 
protein.drop(['Production', 'Import Quantity', 'Stock Variation',
       'Domestic supply quantity', 'Seed', 'Losses', 'Food',
       'Fat supply quantity (g/capita/day)', 'Feed', 'Export Quantity',
       'Processing', 'Other uses', 'Food supply quantity (kg/capita/yr)', 'Food supply (kcal/capita/day)'], axis=1, inplace = True)
protein.head(1)

,ctry_cd,country,item_cd,item,origin,population,Protein supply quantity (g/capita/day)
0,1,Armenia,2511,Wheat and products,vegetal,2977000,30.52


In [36]:
# on retire les NaN
protein = protein[protein['Protein supply quantity (g/capita/day)'].notnull()]

In [37]:
# création de la colomne 'Protein supply (kg/ctry/y)' = 'Protein supply quantity (g/capita/day)' x 'population' x 365 days / 1000 grammes
protein['Protein supply (kg/ctry/y)'] = protein['Protein supply quantity (g/capita/day)'] * protein['population']  * (365 / 1000)

# supression de la variable 'Protein supply quantity (g/capita/day)' 
protein.drop(['Protein supply quantity (g/capita/day)'], axis=1, inplace = True)
protein.head(2)

,ctry_cd,country,item_cd,item,origin,population,Protein supply (kg/ctry/y)
0,1,Armenia,2511,Wheat and products,vegetal,2977000,33163184.6
1,1,Armenia,2513,Barley and products,vegetal,2977000,0.0


# Question 4

### 4.1 -  calculez pour chaque pays et chaque produit le ratio "énergie/poids" en kcal/kg ###
Vous pouvez vérifier la cohérence de votre calcul en comparant ce ratio aux données disponibles sur internet, par exemple en cherchant la valeur calorique d'un oeuf.

Indication : La disponibilité alimentaire en kcal/personne/jour est calculée par la FAO en multipliant la quantité Nouriture (Food) par le ratio énergie/poids (en kcal/kg), puis en le divisant par la population du pays puis par 365. Ici, on vous demande juste de retrouver le ratio énergie/poids que la FAO a utilisé dans son calcul.

#### 4.1.1 - Traitement du data 'Food Supply' en kcal/ctry/y ####

In [38]:
# récupération de Food 2013 de la question 3.1
food2 = food.copy()
food2.head(0)

,ctry_cd,country,item_cd,item,origin,population,Food supply (kcal/ctry/y)


In [39]:
# on retire les éventuels NaN
food2 = food2[food2['Food supply (kcal/ctry/y)'].notnull()]

# nb de lignes du df
rows=len(food2.axes[0])
print(rows)

14241


#### 4.1.2 - Traitement du data 'Food Supply' en kg ####

In [40]:
# récupération du pivot 'data' du Travail Préparatoire TP3
data2 = data.copy()
data2.head(0)

,ctry_cd,country,item_cd,item,origin,population,Domestic supply quantity,Export Quantity,Fat supply quantity (g/capita/day),Feed,...,Food supply (kcal/capita/day),Food supply quantity (kg/capita/yr),Import Quantity,Losses,Other uses,Processing,Production,Protein supply quantity (g/capita/day),Seed,Stock Variation


In [41]:
# on retire les colonnes inutiles
data2.drop(['Production', 'Import Quantity', 'Stock Variation',
       'Domestic supply quantity', 'Seed', 'Losses', 'Food', 'Food supply (kcal/capita/day)',
       'Fat supply quantity (g/capita/day)', 'Feed', 'Export Quantity',
       'Processing', 'Other uses', 'Protein supply quantity (g/capita/day)'], axis=1, inplace = True)

data2.head(1)

,ctry_cd,country,item_cd,item,origin,population,Food supply quantity (kg/capita/yr)
0,1,Armenia,2511,Wheat and products,vegetal,2977000,130.6


In [42]:
# nouvelle colonne produit de 'Food supply quantity (kg/capita/yr)' par 'population'
data2['Food supply (kg/ctry/y)'] =data2['Food supply quantity (kg/capita/yr)'] * data2['population']

# On supprime 'Food supply quantity (kg/capita/yr)'
data2.drop(['Food supply quantity (kg/capita/yr)'], axis=1, inplace = True)

# on retire les NaN
data2 = data2[data2['Food supply (kg/ctry/y)'].notnull()]

# Tri par ordre descendant sur 'Food ratio (kcal/kg)'
data2.sort_values(['Food supply (kg/ctry/y)'], ascending=[True], inplace=True)
data2.head()

,ctry_cd,country,item_cd,item,origin,population,Food supply (kg/ctry/y)
6967,110,Japan,2516,Oats,vegetal,127144000,-245387920.0
14112,229,United Kingdom,2575,Cottonseed Oil,vegetal,63384000,0.0
15374,272,Serbia,2577,Palm Oil,vegetal,9511000,0.0
15369,272,Serbia,2572,Groundnut Oil,vegetal,9511000,0.0
5654,95,Honduras,2775,Aquatic Plants,animal,8096000,0.0


In [43]:
# on regarde le comportement des valeurs
data2['Food supply (kg/ctry/y)'].describe()

count    1.401500e+04
mean     3.479711e+08
std      4.475749e+09
min     -2.453879e+08
25%      2.590500e+05
50%      5.092920e+06
75%      5.229572e+07
max      4.268516e+11
Name: Food supply (kg/ctry/y), dtype: float64

* étrangement nous avons un Min négatif pour [Japan, Oats]. **Est-ce normal ?**

#### 4.1.3 - Merge de 'Food supply (kg/ctry/y)' et de 'Food supply (kcal/ctry/y)' ####

In [44]:
# merge de data2 et food2, inner me semble plus approprié vu que l'on va réaliser un ratio des 2 variables
ratio = pd.merge(data2,food2, how='inner',on=['ctry_cd', 'country', 'item_cd', 'item', 'origin', 'population'])
ratio.head(2)

,ctry_cd,country,item_cd,item,origin,population,Food supply (kg/ctry/y),Food supply (kcal/ctry/y)
0,110,Japan,2516,Oats,vegetal,127144000,-245387920.0,-9.745588e+11
1,229,United Kingdom,2575,Cottonseed Oil,vegetal,63384000,0.0,0.000000e+00


In [45]:
# nombre de lignes du df 'pivot'
rows=len(ratio.axes[0])
print(rows)

13996


#### 4.1.4 - création de 'Food ratio (kcal/kg)'

In [46]:
# création de la colomne 'Food ratio (kcal/kg)' 
ratio['Food ratio (kcal/kg)'] = ratio['Food supply (kcal/ctry/y)'] / ratio['Food supply (kg/ctry/y)']

# on retire les variables inutiles
ratio.drop(['Food supply (kcal/ctry/y)','Food supply (kg/ctry/y)'], axis=1, inplace = True)
ratio.head(2)

,ctry_cd,country,item_cd,item,origin,population,Food ratio (kcal/kg)
0,110,Japan,2516,Oats,vegetal,127144000,3971.502591
1,229,United Kingdom,2575,Cottonseed Oil,vegetal,63384000,NaN


In [47]:
# on regarde le comportement des valeurs
ratio['Food ratio (kcal/kg)'].describe()

count    1.208600e+04
mean              inf
std               NaN
min      0.000000e+00
25%      3.313732e+02
50%      1.184424e+03
75%      3.421875e+03
max               inf
Name: Food ratio (kcal/kg), dtype: float64

* nous avons la valeur inf en max => fruit d'une division 0/x ? 
* std dev = NaN => NaN à suprimer
* min = 0 

In [48]:
# on transforme inf en NaN
ratio = ratio.replace([np.inf], np.nan)
# on retire les NaN
ratio = ratio[ratio['Food ratio (kcal/kg)'].notnull()]
# exclusion des valeurs = 0
ratio = ratio[(ratio['Food ratio (kcal/kg)'] != 0)]

In [49]:
# on regarde le comportement des valeurs
ratio['Food ratio (kcal/kg)'].describe()

count     9876.000000
mean      2963.773927
std       2976.206889
min         82.392777
25%        651.785714
50%       1961.736124
75%       3625.783980
max      21900.000000
Name: Food ratio (kcal/kg), dtype: float64

* le résultat est plus cohérent
* compte-tenu du comportement des valeurs, j'**apporterai personnellement plus d'attention à la médiane qu'à la moyenne** 

#### 4.1.5 - vérifier la cohérence de votre calcul en comparant ce ratio aux données disponibles en cherchant la valeur calorique d'un oeuf.

[Œuf de poule frais entier cru](https://fr.wikipedia.org/wiki/%C5%92uf_(aliment) : Valeur nutritionnelle moyenne pour 100 g (88 % consommables) = 147 kcal
* Eggs (100g) = **1470 kcal/kg**
* Eggs (88 % consommables) = **1294 kcal/kg**

Je devrais trouver un chiffre entre ces deux valeurs

In [50]:
eggs = ratio.copy()

# on sélectionne 'Eggs' dans la variable 'items'
eggs = eggs[(eggs['item'] == 'Eggs')]
eggs.head(2)

,ctry_cd,country,item_cd,item,origin,population,Food ratio (kcal/kg)
3203,55,Dominica,2744,Eggs,animal,72000,1298.418972
3232,188,Saint Kitts and Nevis,2744,Eggs,animal,54000,1348.011364


In [51]:
# on cherche le min, le max, la moyenne et la médiane
eggs['Food ratio (kcal/kg)'].describe()

count     174.000000
mean     1364.817772
std       123.634186
min       912.500000
25%      1298.821628
50%      1384.184507
75%      1417.678148
max      2281.250000
Name: Food ratio (kcal/kg), dtype: float64

La moyenne est de **1365 kcal/kg** et la médiane de **1384 kcal/kg**. <br>
Elles se trouvent toutes les deux entre les 2 valeurs établies au début du point **4.1.5**

### 4.2 - calculez également le pourcentage de protéines de chaque produit ###
En suivant la même méthodologie, calculez également le pourcentage de protéines de chaque produit (pour chaque pays). Ce pourcentage est obtenu en calculant le ratio "poids de protéines/poids total" (attention aux unités utilisées). Vous pouvez vérifier la cohérence de votre calcul en comparant ce ratio aux données disponibles sur internet, par exemple en cherchant la teneur en protéines de l'avoine.

Ce pourcentage est obtenu en calculant le ratio "poids de protéines/poids total" (attention aux unités utilisées). Vous pouvez vérifier la cohérence de votre calcul en comparant ce ratio aux données disponibles sur internet, par exemple en cherchant la teneur en protéines de l'avoine.

#### 4.2.1 - Récupération des df data 'ratio2' et 'protein' ####

In [52]:
# on reprend le df 'data2' de la question 4.1.21ci-dessus
ratio2 = data2.copy()
ratio2.head(0)

,ctry_cd,country,item_cd,item,origin,population,Food supply (kg/ctry/y)


In [53]:
# on reprend le df 'protein' de la question 3.2
pro = protein.copy()
pro.head(0)

,ctry_cd,country,item_cd,item,origin,population,Protein supply (kg/ctry/y)


#### 4.2.2 - Jointure et traitement du data 'protein' en kcal ####

In [54]:
# inner join 
pourprot = pd.merge(ratio2, pro, how = 'inner')
pourprot.head(2)

,ctry_cd,country,item_cd,item,origin,population,Food supply (kg/ctry/y),Protein supply (kg/ctry/y)
0,110,Japan,2516,Oats,vegetal,127144000,-245387920.0,-17170797.2
1,272,Serbia,2577,Palm Oil,vegetal,9511000,0.0,0.0


In [55]:
# Création de la colonne %
pourprot['% proteins'] = pourprot['Protein supply (kg/ctry/y)'] / pourprot['Food supply (kg/ctry/y)'] * 100

# on retire les colonnes inutiles
pourprot.drop(['Protein supply (kg/ctry/y)', 'Food supply (kg/ctry/y)'], axis=1, inplace = True)
pourprot.head(2)

,ctry_cd,country,item_cd,item,origin,population,% proteins
0,110,Japan,2516,Oats,vegetal,127144000,6.997409
1,272,Serbia,2577,Palm Oil,vegetal,9511000,NaN


In [56]:
proteins = pourprot.copy()
# on retire les NaN
proteins = proteins[proteins['% proteins'].notnull()]
# exclusion des valeurs = 0
proteins = proteins[(proteins['% proteins'] != 0)]
proteins.head(3)

,ctry_cd,country,item_cd,item,origin,population,% proteins
0,110,Japan,2516,Oats,vegetal,127144000,6.997409
1331,55,Dominica,2766,Cephalopods,animal,72000,36.500000
1335,188,Saint Kitts and Nevis,2766,Cephalopods,animal,54000,18.250000


In [57]:
# on regarde le comportement des valeurs
proteins['% proteins'].describe()

count    8781.000000
mean        8.086676
std         7.463112
min         0.007100
25%         1.087250
50%         7.300000
75%        12.166667
max        42.583333
Name: % proteins, dtype: float64

#### 4.2.3 vérifiez la cohérence de votre calcul en comparant ce ratio à la teneur en protéines de l'avoine ####

[Avoine déglumée](https://fr.wikipedia.org/wiki/Avoine_cultiv%C3%A9e), valeur nutritionnelle moyenne pour 100 g
* Apport énergétique = 326 kcal
* Protéines = 10,70 g

Le pourcentage de proteines dans l'**avoine déglumée** est donc de l'ordre de 10,7% selon Wikipedia

In [58]:
# on sélectionne 'Oats' dans la variable 'items'
oats = proteins[(proteins['item'] == 'Oats')]
oats.head(1)

,ctry_cd,country,item_cd,item,origin,population,% proteins
0,110,Japan,2516,Oats,vegetal,127144000,6.997409


In [59]:
# on cherche le min, le max, la moyenne et la médiane
oats['% proteins'].describe()

count    117.000000
mean       8.787115
std        1.633425
min        5.087879
25%        8.111111
50%        8.700581
75%        9.125000
max       18.250000
Name: % proteins, dtype: float64

* Il y a un important écart entre min (5%) et max (18%)
* Cependant la moyenne (8,8%) et la médiane (8,7%) sont plutôt proches l'une de l'autre
* En revanche, l'écart est assez important avec les chiffres de Wikipedia (10,7%)

Peut-être pourrions-nous chercher le pourcentage de proteines en fonction de la céréale complète (non déglumée)<br>
L'autre hypotèse serait de vérifier si les variétés locales cultivées ont des valeurs nutritives différentes selon les pays

# Question 5 # 
Citez 5 aliments parmi les 20 aliments les plus caloriques, en utilisant le ratio énergie/poids. 
réaliser pour chaque aliment une moyenne sur les différents pays 
Vous créerez donc une nouvelle table grâce à une agrégation. 
Attention à bien retirer les valeurs égales à 0 afin de ne pas fausser le calcul de la moyenne.
Citez 5 aliments parmi les 20 aliments les plus riches en protéines.

Pour approfondir la réflexion, il est important de se documenter sur la qualité des protéines, notamment sur l'indice PDCAAS. Voici les articles Wikipedia correspondant : français, anglais. Cet aspect n'est pas demandé dans la soutenance, c'est juste pour la culture générale.

### 5.1 - Citez 5 aliments parmi les 20 aliments les plus caloriques, en utilisant le ratio énergie/poids ###

In [60]:
# on reprend le df ratio du point 4.1.4
top5 = ratio.copy()
top5.head()

,ctry_cd,country,item_cd,item,origin,population,Food ratio (kcal/kg)
0,110,Japan,2516,Oats,vegetal,127144000,3971.502591
2000,8,Antigua and Barbuda,2573,Sunflowerseed Oil,vegetal,90000,18250.000000
2009,55,Dominica,2582,Maize Germ Oil,vegetal,72000,12166.666667
2017,8,Antigua and Barbuda,2579,Sesameseed Oil,vegetal,90000,12166.666667
2021,55,Dominica,2580,Olive Oil,vegetal,72000,9125.000000


In [61]:
# comportement des valeurs
top5['Food ratio (kcal/kg)'].describe()

count     9876.000000
mean      2963.773927
std       2976.206889
min         82.392777
25%        651.785714
50%       1961.736124
75%       3625.783980
max      21900.000000
Name: Food ratio (kcal/kg), dtype: float64

In [62]:
# on retire les variables inutiles
top5.drop(['ctry_cd','country','item_cd','population']
          , axis=1, inplace = True)

# Calcul de la moyenne de 'Food ratio (kcal/kg)' pour chaque élément de 'item'
mean = top5.groupby(top5['item']).mean()

# Tri par ordre décroissant sur 'Food ratio (kcal/kg)'
mean.sort_values(['Food ratio (kcal/kg)'], ascending=[False], inplace=True)

mean.head(20)

,Food ratio (kcal/kg)
item,
"Fish, Liver Oil",11809.003531
Sesameseed Oil,10146.064841
Olive Oil,9292.674322
Maize Germ Oil,9257.130389
Ricebran Oil,9206.134101
Rape and Mustard Oil,9149.964104
Groundnut Oil,9118.262843
Sunflowerseed Oil,9027.791629
Palmkernel Oil,9012.117711


**NB :** Ce n'est pas une surprise, les aliments contenant le plus de lipides sont les plus caloriques.<br>
Ici le top 5 : Fish, Liver Oil, Sesameseed Oil, Olive Oil, Maize Germ Oil, Ricebran Oil

### 5.2 - Citez 5 aliments parmi les 20 aliments les plus riches en protéines ###
**NB :** la table a été agrégée antérieurement dans les chapitres 'Travail préparatoire' (TP1, TP2, TP3)

In [63]:
# on reprend le df 'proteins' 4.2.2
proteins.head(1)

,ctry_cd,country,item_cd,item,origin,population,% proteins
0,110,Japan,2516,Oats,vegetal,127144000,6.997409


In [64]:
# on retire les variables inutiles
proteins.drop(['ctry_cd','country','item_cd','population']
          , axis=1, inplace = True)
proteins.head(0)

,item,origin,% proteins


In [65]:
# Calcul de la moyenne de 'Food ratio (kcal/kg)' pour chaque élément de 'item'
meanpro = proteins.groupby(proteins['item']).mean()

# Tri par ordre décroissant sur 'Food ratio (kcal/kg)'
meanpro.sort_values(['% proteins'], ascending=[False], inplace=True)
meanpro.head(20)

,% proteins
item,
Rape and Mustardseed,36.500000
Soyabeans,27.254546
Groundnuts (Shelled Eq),25.091310
Peas,22.423943
"Pulses, Other and products",22.304684
Beans,21.865805
"Oilcrops, Other",18.780182
Sesame seed,18.710715
"Meat, Other",17.848150


* On constatera que les 8 premiers aliments les plus riches en protéines sont des végéteaux. <br>En voici cinq : Soyabeans, Groundnuts (Shelled Eq), Peas, Pulses, Other and products, Beans

* Pour 'Rape and Mustardseed' je ne suis pas convaincu de son utilisation humaine comme source de protéines sans transformation

# Question 6 

Calculez, pour les produits végétaux **uniquement**, la disponibilité intérieure mondiale exprimée en kcal.

### 6.1 - Préparation du dataframe

In [66]:
# on reprend le df 'data' du TP3 qui est le pivot de la variable Element' pour récupérer 'Domestic supply quantity'
datapiv = data.copy()
# on sélectionne 'vegetal' dans la variable 'origin'
datapiv = datapiv[(datapiv['origin'] == 'vegetal')]
datapiv.head(0)

,ctry_cd,country,item_cd,item,origin,population,Domestic supply quantity,Export Quantity,Fat supply quantity (g/capita/day),Feed,...,Food supply (kcal/capita/day),Food supply quantity (kg/capita/yr),Import Quantity,Losses,Other uses,Processing,Production,Protein supply quantity (g/capita/day),Seed,Stock Variation


In [67]:
# on retire les colonnes inutiles
datapiv.drop(['Production', 'Import Quantity', 'Stock Variation',
              'Seed', 'Losses', 'Food', 'Food supply (kcal/capita/day)',
              'Fat supply quantity (g/capita/day)', 'Feed', 'Export Quantity',
              'Processing', 'Other uses', 'Protein supply quantity (g/capita/day)',
              'Food supply quantity (kg/capita/yr)'], axis=1, inplace = True)

# on multiplie 'Domestic supply quantity' par son unité = 1000 tonnes pour avoir la valeur en kg/y
datapiv['Domestic supply quantity'] = datapiv['Domestic supply quantity'] * 1000000

# On renomme les variables
datapiv.columns = ['ctry_cd','country','item_cd','item','origin','population','Domestic supply (kg/y)']

# on retire les éventuels NaN
datapiv = datapiv[datapiv['Domestic supply (kg/y)'].notnull()]
datapiv.head(1)

,ctry_cd,country,item_cd,item,origin,population,Domestic supply (kg/y)
0,1,Armenia,2511,Wheat and products,vegetal,2977000,554000000.0


In [68]:
# disponibilité intérieure mondiale exprimée en kg/y
datapiv['Domestic supply (kg/y)'].sum()

8482244000000.0

In [69]:
# on reprend le df 'ratio' ('Food ratio (kcal/kg)' de l'Ex 4.1
# on retire les éventuels NaN
ratio = ratio[ratio['Food ratio (kcal/kg)'].notnull()]
ratio.head(1)

,ctry_cd,country,item_cd,item,origin,population,Food ratio (kcal/kg)
0,110,Japan,2516,Oats,vegetal,127144000,3971.502591


In [70]:
# comportement des valeurs
ratio['Food ratio (kcal/kg)'].describe()

count     9876.000000
mean      2963.773927
std       2976.206889
min         82.392777
25%        651.785714
50%       1961.736124
75%       3625.783980
max      21900.000000
Name: Food ratio (kcal/kg), dtype: float64

### 6.2 merge des 2 df

In [71]:
# merge de datapiv et food  
dim = pd.merge(datapiv, ratio, on=['ctry_cd', 'country', 'item_cd', 'item', 'origin', 'population'])
dim.head(1)

,ctry_cd,country,item_cd,item,origin,population,Domestic supply (kg/y),Food ratio (kcal/kg)
0,1,Armenia,2511,Wheat and products,vegetal,2977000,554000000.0,2861.8683


In [72]:
# création d'une nouvelle colonne 'Domestic supply kcal' 
dim['Domestic supply (kcal/y)'] = dim['Domestic supply (kg/y)'] * dim['Food ratio (kcal/kg)']

# on retire les colonnes inutiles
dim.drop(['Domestic supply (kg/y)','Food ratio (kcal/kg)'], axis=1, inplace = True)

# on ne garde que 'origin' = 'vegetal'
dimveg = dim[(dim['origin'] == 'vegetal')]
dimveg.head(2)

,ctry_cd,country,item_cd,item,origin,population,Domestic supply (kcal/y)
0,1,Armenia,2511,Wheat and products,vegetal,2977000,1.585475e+12
1,1,Armenia,2515,Rye and products,vegetal,2977000,3.041667e+09


In [73]:
# disponibilité intérieure mondiale exprimée en kcal/year
dimveg['Domestic supply (kcal/y)'].sum()

1.2166421524173388e+16

 # Question 7 #

Combien d'humains pourraient être nourris si toute la disponibilité intérieure mondiale de produits végétaux était utilisée pour de la nourriture ? Donnez les résultats en termes de calories, puis de protéines, et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

Renseignez-vous sur la recommandation (de la FAO si possible) concernant le nombre de calories ainsi que sur le nombre de protéines par jour nécessaire à un être humain (Allez zou, on recherche sur internet ! ;) ). 4.99e+12

### 7.1 - Travail préparatoire, documentation

Selon la [MÉTHODOLOGIE DE LA FAO POUR MESURER LA PRÉVALENCE DE LA SOUS-ALIMENTATION](http://www.fao.org/fileadmin/templates/ess/documents/food_security_statistics/metadata/methologie_sousalimentation.pdf) page 11, le besoin énergétique minimum moyen (adulte/enfant, homme/femme, femme enceinte) est de **1680 kcal/personne/jour** alors que la valeur proposée par [Wikipedia](https://fr.wikipedia.org/wiki/Apports_journaliers_recommand%C3%A9s) est de **2000 kcal/personne/jour**


Quant au besoin minimum en protéines au niveau mondial, il est introuvable dans les rapports de la FAO et semble extrêmement complexe à évaluer : pyramide des âges, sexe, femmes allaitantes ou enceintes, poids moyen par tranche d'âge, hétérogénéité des valeurs du même aliment selon les pays, etc.

Je me contenterai donc de prendre la valeur proposée par [Wikipedia](https://fr.wikipedia.org/wiki/Apports_journaliers_recommand%C3%A9s) soit **50 grammes**

**Rapports de la FAO**<br>
http://www.fao.org/docrep/003/aa040e/AA040E00.htm#TOC
http://www.fao.org/fileadmin/templates/ess/documents/food_security_statistics/metadata/methodologie_sousalimentation.pdf
http://apps.who.int/iris/bitstream/handle/10665/62734/WHO_TRS_724_%28part1%29_fre.pdf?sequence=1&isAllowed=y http://apps.who.int/iris/bitstream/handle/10665/62734/WHO_TRS_724_%28part2%29_fre.pdf?sequence=2&isAllowed=y

### 7.2 - Combien d'humains pourraient être nourris si toute la disponibilité intérieure mondiale de produits végétaux était utilisée pour de la nourriture ? Donnez les résultats en termes de calories

In [74]:
humkcal = dimveg.copy()
humkcal.head(2)

,ctry_cd,country,item_cd,item,origin,population,Domestic supply (kcal/y)
0,1,Armenia,2511,Wheat and products,vegetal,2977000,1.585475e+12
1,1,Armenia,2515,Rye and products,vegetal,2977000,3.041667e+09


In [75]:
# disponibilité intérieure mondiale exprimée en kcal/j pour une ration de 1680 kcal/j
people = humkcal['Domestic supply (kcal/y)'].sum() / (365 * 1680)
print(people)

19840870065.51433


In [76]:
# disponibilité intérieure mondiale exprimée en kcal/j pour une ration de 2000 kcal/j
people = humkcal['Domestic supply (kcal/y)'].sum() / (365 * 2000)
print(people)

16666330855.032038


* entre 16,7 et 19,8 milliards d'humains pourraient être nourris avec des poduits végétaux selon la ration minimale choisie

# Question 8 #
Combien d'humains pourraient être nourris si toute la disponibilité alimentaire en produits végétaux (Food) la nourriture végétale destinée aux animaux (Feed) et les pertes de produits végétaux (Waste erreur de l'énoncé le nom correct est Losses) étaient utilisés pour de la nourriture ? 
* Donnez les résultats en termes de calories, 
* puis de protéines, 
* et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

### 8.1 - Donnez les résultats en termes de calories

#### 8.1.1 - on reprend le df 'data' du TP3 

In [77]:
# on reprend le df 'data' du TP3 qui est le pivot de la variable Element' 
datapiv2 = data.copy()
datapiv2.head(1)

,ctry_cd,country,item_cd,item,origin,population,Domestic supply quantity,Export Quantity,Fat supply quantity (g/capita/day),Feed,...,Food supply (kcal/capita/day),Food supply quantity (kg/capita/yr),Import Quantity,Losses,Other uses,Processing,Production,Protein supply quantity (g/capita/day),Seed,Stock Variation
0,1,Armenia,2511,Wheat and products,vegetal,2977000,554.0,1.0,3.6,93.0,...,1024.0,130.6,361.0,32.0,0.0,10.0,312.0,30.52,30.0,-118.0


In [78]:
# on retire les colonnes inutiles
datapiv2.drop(['Production', 'Import Quantity', 'Stock Variation',
       'Seed', 'Food supply (kcal/capita/day)','Domestic supply quantity',
       'Fat supply quantity (g/capita/day)', 'Export Quantity',
        'Processing', 'Other uses', 'Protein supply quantity (g/capita/day)','Food supply quantity (kg/capita/yr)'], axis=1, inplace = True)

datapiv2.head(1)

,ctry_cd,country,item_cd,item,origin,population,Feed,Food,Losses
0,1,Armenia,2511,Wheat and products,vegetal,2977000,93.0,389.0,32.0


In [79]:
# on ne garde que 'origin' = 'vegetal'
datapiv2 = datapiv2[(datapiv2['origin'] == 'vegetal')]

# check valeurs uniques de 'origin'
datapiv2['origin'].unique()

array(['vegetal'], dtype=object)

In [80]:
# Il faut créer une nouvelle colonne somme de Food + Feed + Losses x unit = 1000 tonnes
datapiv2['FFL (kg/y)'] = datapiv2['Food'] + datapiv2['Feed'] + datapiv2['Losses']
datapiv2['FFL (kg/y)'] = datapiv2['FFL (kg/y)'] * 1000000

# on retire les colonnes inutiles
datapiv2.drop(['Feed','Food','Losses'], axis=1, inplace = True)
datapiv2.head(2)

,ctry_cd,country,item_cd,item,origin,population,FFL (kg/y)
0,1,Armenia,2511,Wheat and products,vegetal,2977000,514000000.0
1,1,Armenia,2513,Barley and products,vegetal,2977000,152000000.0


In [81]:
# comportement des valeurs
datapiv2['FFL (kg/y)'].describe()

count    1.196000e+03
mean     2.664390e+09
std      1.726390e+10
min      0.000000e+00
25%      3.200000e+07
50%      2.370000e+08
75%      1.169000e+09
max      5.000700e+11
Name: FFL (kg/y), dtype: float64

#### 8.1.2 - on reprend le df 'ratio' de l'ex 4.1

In [82]:
# on reprend le df 'ratio' ('Food ratio (kcal/kg)' de l'Ex 4.1
frkcal=ratio.copy()

# on ne garde que 'origin' = 'vegetal'
frkcal = frkcal[(frkcal['origin'] == 'vegetal')]

frkcal.head(2)

,ctry_cd,country,item_cd,item,origin,population,Food ratio (kcal/kg)
0,110,Japan,2516,Oats,vegetal,127144000,3971.502591
2000,8,Antigua and Barbuda,2573,Sunflowerseed Oil,vegetal,90000,18250.000000


In [83]:
# comportement des valeurs
frkcal['Food ratio (kcal/kg)'].describe()

count     7404.000000
mean      3248.616689
std       3113.698165
min         82.392777
25%        599.726160
50%       2744.968062
75%       3872.216085
max      21900.000000
Name: Food ratio (kcal/kg), dtype: float64

In [84]:
# merge de 'frkcal' et de 'datapiv2' sur la clé 'item'
ffl = pd.merge(frkcal, datapiv2, on=['ctry_cd','country','item_cd','item','origin','population'])


ffl.head(1)

,ctry_cd,country,item_cd,item,origin,population,Food ratio (kcal/kg),FFL (kg/y)
0,110,Japan,2516,Oats,vegetal,127144000,3971.502591,NaN


In [85]:
# comportement des valeurs
ffl['Food ratio (kcal/kg)'].describe()

count     7404.000000
mean      3248.616689
std       3113.698165
min         82.392777
25%        599.726160
50%       2744.968062
75%       3872.216085
max      21900.000000
Name: Food ratio (kcal/kg), dtype: float64

In [86]:
# Il faut créer une nouvelle colonne produit de Food ratio par FFL
ffl['FFL (kcal/y)'] = ffl['Food ratio (kcal/kg)'] * ffl['FFL (kg/y)']
ffl.head(2)

,ctry_cd,country,item_cd,item,origin,population,Food ratio (kcal/kg),FFL (kg/y),FFL (kcal/y)
0,110,Japan,2516,Oats,vegetal,127144000,3971.502591,NaN,NaN
1,8,Antigua and Barbuda,2573,Sunflowerseed Oil,vegetal,90000,18250.000000,NaN,NaN


In [87]:
# population FFL (kcal/day)
r1 = ffl['FFL (kcal/y)'].sum() / 365
print(r1)

16722889611456.137


In [88]:
# ici avec le calcul de la FAO = 1680 kcal/day
r2 = r1 / 1680
print(r2)

9954100959.20008


In [89]:
# ici avec le calcul de Wikipedia = 2000 kcal/day
r3 = r1 / 2000
print(r3)

8361444805.728068


nous pourrions nourrir entre 8,4 et 10,0 milliards d'humains

### 8.2 - Donnez les résultats en termes de protéines

In [90]:
# 4.1.2
ccc= pourprot.copy()
ccc.head(2)

,ctry_cd,country,item_cd,item,origin,population,% proteins
0,110,Japan,2516,Oats,vegetal,127144000,6.997409
1,272,Serbia,2577,Palm Oil,vegetal,9511000,NaN


In [91]:
# on retire les NaN
ccc = ccc[ccc['% proteins'].notnull()]

# comportement des valeurs
ccc['% proteins'].describe()

count    10064.000000
mean         7.055753
std          7.474716
min          0.000000
25%          0.604605
50%          5.645438
75%         11.397840
max         42.583333
Name: % proteins, dtype: float64

In [92]:
# 8.1
prote = datapiv2.copy()
prote.head(2)

,ctry_cd,country,item_cd,item,origin,population,FFL (kg/y)
0,1,Armenia,2511,Wheat and products,vegetal,2977000,514000000.0
1,1,Armenia,2513,Barley and products,vegetal,2977000,152000000.0


In [93]:
# fusion 
fflpro = pd.merge(ccc, prote, how='inner', on=['ctry_cd','country','item_cd','item','origin','population'])
fflpro.head(2)

,ctry_cd,country,item_cd,item,origin,population,% proteins,FFL (kg/y)
0,110,Japan,2516,Oats,vegetal,127144000,6.997409,NaN
1,83,Kiribati,2601,Tomatoes and products,vegetal,102000,0.000000,NaN


In [94]:
# Il faut créer une nouvelle colonne 
fflpro['FFL proteins (kg/y)'] = fflpro['% proteins'] * fflpro['FFL (kg/y)'] / 100
# on retire les NaN
fflpro = fflpro[fflpro['FFL proteins (kg/y)'].notnull()]
fflpro.head(2)

,ctry_cd,country,item_cd,item,origin,population,% proteins,FFL (kg/y),FFL proteins (kg/y)
33,155,Vanuatu,2514,Maize and products,vegetal,253000,9.125,1000000.0,91250.0
72,273,Montenegro,2513,Barley and products,vegetal,621000,0.000,0.0,0.0


In [95]:
# FFF proteins (kg/day)
z1 = fflpro['FFL proteins (kg/y)'].sum() / 365
print(z1)

441224878.45959324


In [96]:
# ici avec le calcul Wikipedia ration minimum quotidienne de protéines = 50g/jour
z2 = z1 / 0.05
print(z2)

8824497569.191864


* nous pourrions fournir 50g de proteines par jour à 8,8 milliards d'humains

### 8.3 - Exprimez ces 2 résultats en pourcentage de la population mondiale ###
Je choisis la population 2013 de la FAO de l'ex 1.4 soit 6997326000 

In [97]:
# % de la population mondiale pouvant être nourrie avec des végétaux pour FFL (kcal/day)
8361444806 / 6997326000 * 100

119.4948585502519

In [98]:
# % de la population mondiale pouvant obtenir une ration minimum de protéines végétales FFL proteins (kg/day)
8824497569 / 6997326000 * 100

126.11242593242046

# Question 9 #
Combien d'humains pourraient être nourris avec la disponibilité alimentaire (Food Supply) mondiale ? 
* Donnez les résultats en termes de calories
* puis de protéines
* exprimez ces 2 résultats en pourcentage de la population mondiale

### 9.1 - Donnez les résultats en termes de calories

In [99]:
# Import du fichier supply du TP2 fichier séparé
# Agregat China:'ctry_cd' 351 = 96 + 128 + 41 + 214 exclu
# origine = vegetal + animal
c = pd.read_csv('data/P3_TP2_2013.csv', index_col=0)
c.head(0)

,ctry_cd,country,item_cd,item,origin,population,Fat supply quantity (g/capita/day),Food supply (kcal/capita/day),Food supply quantity (kg/capita/yr),Protein supply quantity (g/capita/day)


In [100]:
# on retire les colonnes inutiles
c.drop(['Fat supply quantity (g/capita/day)','Food supply quantity (kg/capita/yr)',
        'Protein supply quantity (g/capita/day)'],axis=1, inplace = True)
c.head(1)

,ctry_cd,country,item_cd,item,origin,population,Food supply (kcal/capita/day)
0,1,Armenia,2511,Wheat and products,vegetal,2977000,1024.0


In [101]:
# nouvelle colonne FS ctry item (kcal/day)
c['FS ctry item (kcal/day)'] = c['population'] * c['Food supply (kcal/capita/day)']
c.head(1)

,ctry_cd,country,item_cd,item,origin,population,Food supply (kcal/capita/day),FS ctry item (kcal/day)
0,1,Armenia,2511,Wheat and products,vegetal,2977000,1024.0,3.048448e+09


In [102]:
# somme de 'FS ctry item (kcal/day)' divisé par la ration minimale de la FAO = 1680 kcal/day
c = c['FS ctry item (kcal/day)'].sum() / 1680
print(c)

12008942302.97619


* environ 12 milliards d'être humains pourraient être nourris à hauteur de 1680 kcal/jour avec Food Supply

### 9.2 - Donnez les résultats en termes de proteines

In [103]:
# Import du fichier supply du TP2 fichier séparé
# Agregat China:'ctry_cd' 351 = 96 + 128 + 41 + 214 exclu
# origine = vegetal + animal
p = pd.read_csv('data/P3_TP2_2013.csv', index_col=0)
p.head(0)

,ctry_cd,country,item_cd,item,origin,population,Fat supply quantity (g/capita/day),Food supply (kcal/capita/day),Food supply quantity (kg/capita/yr),Protein supply quantity (g/capita/day)


In [104]:
# on retire les colonnes inutiles
p.drop(['Fat supply quantity (g/capita/day)','Food supply (kcal/capita/day)','Food supply quantity (kg/capita/yr)'],axis=1, inplace = True)
p.head(1)

,ctry_cd,country,item_cd,item,origin,population,Protein supply quantity (g/capita/day)
0,1,Armenia,2511,Wheat and products,vegetal,2977000,30.52


In [105]:
# nouvelle colonne FS prot ctry item (g/day)
p['FS prot ctry item (g/day)'] = p['population'] * p['Protein supply quantity (g/capita/day)']
p.head(1)

,ctry_cd,country,item_cd,item,origin,population,Protein supply quantity (g/capita/day),FS prot ctry item (g/day)
0,1,Armenia,2511,Wheat and products,vegetal,2977000,30.52,90858040.0


In [106]:
# somme de 'FS prot ctry item (g/day' divisée par la ration minimale de proteines Wikipedia = 50 g/day
p = p['FS prot ctry item (g/day)'].sum() / 50
print(p)

11356667401.4


* environ 11,4 milliards d'être humains pourraient recevoir une ration de 50g de protéines/jour avec Food Supply

### 9.3 - Exprimez ces 2 résultats en pourcentage de la population mondiale ###
Je choisis la population 2013 de la FAO de l'ex 1.4 soit 6997326000 

In [107]:
# % de la population mondiale pouvant être nourrie à hauteur de 1680 kcal/jour avec Food Supply
12008942303 / 6997326000 * 100

171.6218781717473

In [108]:
# % de la population mondiale pouvant être nourrie à hauteur de 50g de protéines/jour avec Food Supply
11356667401 / 6997326000 * 100

162.30010436844017

# Question 10 #
A partir des données téléchargées qui concernent la sous-nutrition, répondez à cette question : 
* Quelle proportion de la population mondiale est considérée comme étant en sous-nutrition ?
* Établissez la liste des produits (ainsi que leur code) considéré comme des céréales selon la FAO.
* Repérez dans vos données les informations concernant les céréales (par exemple en créant une colonne de type booléen nommée "is_cereal").

### 10.1 - Data wrangling

In [109]:
# Import des données de la FAO
starve = pd.read_csv('data/fs_undernourished_2013.csv')
starve.head(1)

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,FS,Suite of Food Security Indicators,2,Afghanistan,6132,Value,210011,Number of people undernourished (million) (3-y...,20122014,2012-2014,millions,8.1,F,FAO estimate


In [110]:
#check de Flag Description
starve['Item'].unique()

array(['Number of people undernourished (million) (3-year average)'],
      dtype=object)

In [111]:
starve['Area'].count()

204

In [112]:
# On exclut l'agrégat "China" :
starve = starve[(starve['Area Code'] != 351)]
starve['Area'].count()

203

In [113]:
#check de Flag Description
starve['Flag Description'].unique()

array(['FAO estimate', 'Data not available'], dtype=object)

In [114]:
# on retire 'Data not available' de 'Flag Description'
starve = starve[(starve['Flag Description'] != 'Data not available')]
#check de Flag Description
starve['Flag Description'].unique()

array(['FAO estimate'], dtype=object)

In [115]:
# Recherche pour 'Value' des lignes contenant l'opérateur '<'
st = starve[starve['Value'].str.contains('<')]
st

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
9,FS,Suite of Food Security Indicators,10,Australia,6132,Value,210011,Number of people undernourished (million) (3-y...,20122014,2012-2014,millions,<0.6,F,FAO estimate
10,FS,Suite of Food Security Indicators,11,Austria,6132,Value,210011,Number of people undernourished (million) (3-y...,20122014,2012-2014,millions,<0.2,F,FAO estimate
11,FS,Suite of Food Security Indicators,52,Azerbaijan,6132,Value,210011,Number of people undernourished (million) (3-y...,20122014,2012-2014,millions,<0.2,F,FAO estimate
15,FS,Suite of Food Security Indicators,14,Barbados,6132,Value,210011,Number of people undernourished (million) (3-y...,20122014,2012-2014,millions,<0.1,F,FAO estimate
16,FS,Suite of Food Security Indicators,57,Belarus,6132,Value,210011,Number of people undernourished (million) (3-y...,20122014,2012-2014,millions,<0.2,F,FAO estimate
17,FS,Suite of Food Security Indicators,255,Belgium,6132,Value,210011,Number of people undernourished (million) (3-y...,20122014,2012-2014,millions,<0.3,F,FAO estimate
18,FS,Suite of Food Security Indicators,23,Belize,6132,Value,210011,Number of people undernourished (million) (3-y...,20122014,2012-2014,millions,<0.1,F,FAO estimate
23,FS,Suite of Food Security Indicators,80,Bosnia and Herzegovina,6132,Value,210011,Number of people undernourished (million) (3-y...,20122014,2012-2014,millions,<0.1,F,FAO estimate
25,FS,Suite of Food Security Indicators,21,Brazil,6132,Value,210011,Number of people undernourished (million) (3-y...,20122014,2012-2014,millions,<5.1,F,FAO estimate
26,FS,Suite of Food Security Indicators,26,Brunei Darussalam,6132,Value,210011,Number of people undernourished (million) (3-y...,20122014,2012-2014,millions,<0.1,F,FAO estimate


* 70 lignes de 'Value' ont une estimation FAO exprimée sous la forme v<n
* nous n'avons pas d'autres chiffres, donc je garde la valeur maxi et retire < de manière à pouvoir effectuer les opérations.
* je devrais donc inclure < à l'intitulé du résultat 

In [116]:
# je retire l'opérateur '<' dans les valeurs de 'Value' qui le contiennent
starve['Value'].replace(regex=True,inplace=True,to_replace=r'\<',value=r'')
starve.head(1)

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,FS,Suite of Food Security Indicators,2,Afghanistan,6132,Value,210011,Number of people undernourished (million) (3-y...,20122014,2012-2014,millions,8.1,F,FAO estimate


In [117]:
# transformation 'Value' = float
starve['Value'] = starve['Value'].astype(float)
# comportement de 'Value'
starve['Value'].dtypes

dtype('float64')

In [118]:
st = starve.copy()
# je multiplie 'Value' par son unité = 1.000.000
st['Value'] = st['Value'] * 1000000
st.head(1)

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,FS,Suite of Food Security Indicators,2,Afghanistan,6132,Value,210011,Number of people undernourished (million) (3-y...,20122014,2012-2014,millions,8100000.0,F,FAO estimate


###  10.2 - Quelle proportion de la population mondiale est considérée comme étant en sous-nutrition ?

In [119]:
# résultat
st['Value'].sum()

782500000.0

In [120]:
# résultat
st['Value'].count()

166

In [121]:
# pourcentage de la population mondiale (chiffres de la FAO 2013)
782500000 / 6997326000  * 100

11.182843274702364

* Le nombre de personnes en sous-nutrition est inférieur à 782,5 millions pour les 166 pays pour lesquels la FAO a des valeurs
* soit 11,2% ramené à la population mondiale de la FAO en 2013

# Question 11

### 11.1 - Établissez la liste des produits (ainsi que leur code) considéré comme des céréales selon la FAO

In [122]:
# Import des données de la FAO
cereals = pd.read_csv('data/fb_cereals_2013.csv')
cereals.head(1)

,Domain Code,Domain,Country Code,Country,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,FBS,Food Balance Sheets,5000,World,5511,Production,2511,Wheat and products,2013,2013,1000 tonnes,708443.0,A,"Aggregate, may include official, semi-official..."


In [123]:
# Valeurs uniques de 'item'
cereals['Item'].unique()

array(['Wheat and products', 'Rice (Milled Equivalent)',
       'Barley and products', 'Maize and products', 'Rye and products',
       'Oats', 'Millet and products', 'Sorghum and products',
       'Cereals, Other'], dtype=object)

In [124]:
# on retire les colonnes inutiles
cereals.drop(['Domain Code','Domain','Country Code','Country',
              'Element Code','Element','Year Code','Year','Unit','Value','Flag','Flag Description',],axis=1, inplace = True)
cereals.head(1)

,Item Code,Item
0,2511,Wheat and products


In [125]:
# Valeurs uniques de 'Item Code'
cereals['Item Code'].unique()

array([2511, 2805, 2513, 2514, 2515, 2516, 2517, 2518, 2520], dtype=int64)

### 10.3 - Repérez dans vos données les informations concernant les céréales (par exemple en créant une colonne de type booléen nommée "is_cereal")

In [126]:
# on reprend 'data' du TP3
datcer = data.copy()
datcer.head(1)

,ctry_cd,country,item_cd,item,origin,population,Domestic supply quantity,Export Quantity,Fat supply quantity (g/capita/day),Feed,...,Food supply (kcal/capita/day),Food supply quantity (kg/capita/yr),Import Quantity,Losses,Other uses,Processing,Production,Protein supply quantity (g/capita/day),Seed,Stock Variation
0,1,Armenia,2511,Wheat and products,vegetal,2977000,554.0,1.0,3.6,93.0,...,1024.0,130.6,361.0,32.0,0.0,10.0,312.0,30.52,30.0,-118.0


In [127]:
# Valeurs uniques de 'Item Code'
datcer['origin'].unique()

array(['vegetal', 'animal'], dtype=object)

In [128]:
# on sélectionne les valeurs 'is_cereal'
ic = data.copy()
ic = ic[(ic['item_cd'].isin([2511, 2805, 2513, 2514, 2515, 2516, 2517, 2518, 2520]))]
ic.head(2)

,ctry_cd,country,item_cd,item,origin,population,Domestic supply quantity,Export Quantity,Fat supply quantity (g/capita/day),Feed,...,Food supply (kcal/capita/day),Food supply quantity (kg/capita/yr),Import Quantity,Losses,Other uses,Processing,Production,Protein supply quantity (g/capita/day),Seed,Stock Variation
0,1,Armenia,2511,Wheat and products,vegetal,2977000,554.0,1.0,3.6,93.0,...,1024.0,130.6,361.0,32.0,0.0,10.0,312.0,30.52,30.0,-118.0
1,1,Armenia,2513,Barley and products,vegetal,2977000,198.0,0.0,0.0,137.0,...,0.0,0.0,9.0,15.0,26.0,7.0,189.0,0.00,14.0,0.0


In [129]:
# on sélectionne les valeurs 'is not cereal'
inc = data.copy()
inc = inc[~(inc['item_cd'].isin([2511, 2805, 2513, 2514, 2515, 2516, 2517, 2518, 2520]))]
inc.head(1)

,ctry_cd,country,item_cd,item,origin,population,Domestic supply quantity,Export Quantity,Fat supply quantity (g/capita/day),Feed,...,Food supply (kcal/capita/day),Food supply quantity (kg/capita/yr),Import Quantity,Losses,Other uses,Processing,Production,Protein supply quantity (g/capita/day),Seed,Stock Variation
8,1,Armenia,2531,Potatoes and products,vegetal,2977000,672.0,9.0,0.12,225.0,...,82.0,45.1,5.0,96.0,16.0,78.0,661.0,1.94,123.0,15.0


In [130]:
# Ajout de la variable origin
ic['is_cereal'] = True
inc['is_cereal'] = False

# On regroupe veg et ani en un unique dataframe, via une union
tp = ic.append(inc)
tp.head(1)

,ctry_cd,country,item_cd,item,origin,population,Domestic supply quantity,Export Quantity,Fat supply quantity (g/capita/day),Feed,...,Food supply quantity (kg/capita/yr),Import Quantity,Losses,Other uses,Processing,Production,Protein supply quantity (g/capita/day),Seed,Stock Variation,is_cereal
0,1,Armenia,2511,Wheat and products,vegetal,2977000,554.0,1.0,3.6,93.0,...,130.6,361.0,32.0,0.0,10.0,312.0,30.52,30.0,-118.0,True


# Question 11 #

* En ne prenant en compte que les céréales destinées à l'alimentation (humaine et animale), quelle proportion (en termes de poids) est destinée à l'alimentation animale ?
* Sélectionnez parmi les données des bilans alimentaires les informations relatives aux pays dans lesquels la FAO recense des personnes en sous-nutrition, pour une année choisie. Repérez les 15 produits les plus exportés par ce groupe de pays sur l'année choisie.
* Parmi les données des bilans alimentaires au niveau mondial, sélectionnez les 200 plus grandes importations de ces produits (1 importation = une quantité d'un produit donné importée par un pays donné sur l'année choisie)
* Groupez ces importations par produit, afin d'avoir une table contenant 1 ligne pour chacun des 15 produits. Ensuite, calculez pour chaque produit les 2 quantités suivantes :
* * le ratio entre la quantité destinés aux "Autres utilisations" (Other uses) et la disponibilité intérieure.
* * le ratio entre la quantité destinée à la nourriture animale et la quantité destinée à la nourriture (animale + humaine)

### 11.1 - En ne prenant en compte que les céréales destinées à l'alimentation (humaine et animale), quelle proportion (en termes de poids) est destinée à l'alimentation animale ?

In [131]:
# on reprend le df 'ic' de l'ex 10.3
anhu = ic.copy()
anhu.head(1)

,ctry_cd,country,item_cd,item,origin,population,Domestic supply quantity,Export Quantity,Fat supply quantity (g/capita/day),Feed,...,Food supply quantity (kg/capita/yr),Import Quantity,Losses,Other uses,Processing,Production,Protein supply quantity (g/capita/day),Seed,Stock Variation,is_cereal
0,1,Armenia,2511,Wheat and products,vegetal,2977000,554.0,1.0,3.6,93.0,...,130.6,361.0,32.0,0.0,10.0,312.0,30.52,30.0,-118.0,True


In [132]:
# check des valeurs uniques de 'is_cereal'
anhu['is_cereal'].unique()

array([ True])

In [133]:
# on retire les colonnes inutiles
anhu.drop(['Production', 'Import Quantity', 'Stock Variation',
       'Seed', 'Food supply (kcal/capita/day)','Domestic supply quantity',
       'Fat supply quantity (g/capita/day)', 'Export Quantity',
        'Processing', 'Other uses', 'Protein supply quantity (g/capita/day)','Food supply quantity (kg/capita/yr)','Losses','is_cereal'], axis=1, inplace = True)
anhu.head(1)

,ctry_cd,country,item_cd,item,origin,population,Feed,Food
0,1,Armenia,2511,Wheat and products,vegetal,2977000,93.0,389.0


In [134]:
# Céréales mondialement destinées à la nutrition animale (1000 tonnes)
anhu['Feed'].sum()

873535.0

In [135]:
# Céréales mondialement destinées à la nutrition humaine (1000 tonnes)
anhu['Food'].sum()

1029010.0

In [136]:
# total
873535 + 1029010

1902545

In [137]:
# % de Céréales mondialement destinées à la nutrition animale (Feed + Food)
873535 / 1902545 * 100

45.91402568664604

* près de 46% des Céréales (Feed + Food) sont destinées à la nutrition animale

### 11.2 - Sélectionnez parmi les données des bilans alimentaires les informations relatives aux pays dans lesquels la FAO recense des personnes en sous-nutrition, pour une année choisie.

#### 11.2.1 - reprise du df 'st' de l'ex 10.1

In [138]:
# reprise du df 'st' de l'ex 10.1
snba = st.copy()
snba.head(0)

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description


In [139]:
# on retire les colonnes inutiles
snba.drop(['Domain Code', 'Domain','Area','Element Code','Element','Item Code','Item','Year Code','Year',
       'Unit','Flag', 'Flag Description'], axis=1, inplace = True)

# On renomme les variables de snba
snba.columns = ['ctry_cd','undernourished']
snba.head(1)

,ctry_cd,undernourished
0,2,8100000.0


In [140]:
# nb de pays
snba['ctry_cd'].count()

166

In [141]:
# check
# on retire les NaN
snba = snba[snba['undernourished'].notnull()]
# on élimine undernourished' = 0
snba = snba[(snba['undernourished'] != 0)]
# count
snba['undernourished'].count()

166

#### 11.2.2 - reprise du df 'data' du TP3

In [142]:
# on reprend 'data' du TP3
snexp = data.copy()
snexp.head(0)

,ctry_cd,country,item_cd,item,origin,population,Domestic supply quantity,Export Quantity,Fat supply quantity (g/capita/day),Feed,...,Food supply (kcal/capita/day),Food supply quantity (kg/capita/yr),Import Quantity,Losses,Other uses,Processing,Production,Protein supply quantity (g/capita/day),Seed,Stock Variation


In [143]:
# on retire les colonnes inutiles
snexp.drop(['Production', 'Import Quantity', 'Stock Variation',
       'Seed', 'Food supply (kcal/capita/day)','Domestic supply quantity',
       'Fat supply quantity (g/capita/day)',
        'Processing', 'Other uses', 'Protein supply quantity (g/capita/day)','Food supply quantity (kg/capita/yr)','Losses','Feed','Food'], axis=1, inplace = True)

# on retire les NaN
snexp = snexp[snexp['Export Quantity'].notnull()]
snexp.head(1)

,ctry_cd,country,item_cd,item,origin,population,Export Quantity
0,1,Armenia,2511,Wheat and products,vegetal,2977000,1.0


In [144]:
# fusion de 'snba et de snexp' sur la clé 'ctry_cd'
consdf = pd.merge(snba, snexp, on='ctry_cd')
consdf.head(1)

,ctry_cd,undernourished,country,item_cd,item,origin,population,Export Quantity
0,2,8100000.0,Afghanistan,2514,Maize and products,vegetal,30552000,0.0


In [145]:
# count
consdf['undernourished'].count()

11908

In [146]:
# check 
# on retire les NaN
consdf = consdf[consdf['undernourished'].notnull()]
# on élimine undernourished' = 0
consdf = consdf[(consdf['undernourished'] != 0)]
# count
consdf['undernourished'].count()

11908

#### 11.2.3 - Repérez les 15 produits les plus exportés par ce groupe de pays sur l'année choisie.

In [147]:
# on retire les variables inutiles
consdf.drop(['ctry_cd','undernourished','country','item_cd','origin','population']
          , axis=1, inplace = True)
consdf.head(0)

,item,Export Quantity


In [148]:
# Calcul de la somme de 'Export Quantity' pour chaque élément de 'item'
grouped = consdf.groupby(consdf['item']).sum()

# Tri par ordre décroissant sur 'Export Quantity'
grouped.sort_values(['Export Quantity'], ascending=[False], inplace=True)
grouped.head(15)

,Export Quantity
item,
Wheat and products,198045.0
Maize and products,128654.0
Milk - Excluding Butter,125090.0
Soyabeans,106899.0
Sugar (Raw Equivalent),68286.0
Palm Oil,50673.0
"Vegetables, Other",44499.0
Barley and products,41460.0
Rice (Milled Equivalent),41444.0


* voici les 15 produits les plus exportés (1000 tonnes) par le groupe de pays disposant d'une population en sous-nutrition en 2013

### 11.3 - Parmi les données des bilans alimentaires au niveau mondial, sélectionnez les 200 plus grandes importations de ces produits (1 importation = une quantité d'un produit donné importée par un pays donné sur l'année choisie)

In [149]:
# on reprend le df 'data' du TP3
imp = data.copy()
# on retire les colonnes inutiles
imp.drop(['Production', 'Export Quantity', 'Stock Variation','Seed', 'Food supply (kcal/capita/day)',
          'Fat supply quantity (g/capita/day)','Processing',
          'Protein supply quantity (g/capita/day)','Food supply quantity (kg/capita/yr)','Losses',], axis=1, inplace = True)
imp.head(0)

,ctry_cd,country,item_cd,item,origin,population,Domestic supply quantity,Feed,Food,Import Quantity,Other uses


In [150]:
# on ne sélectionne les éléments
imp2 = imp.copy()
imp2 = imp2[imp2['item'].isin(['Wheat and products', 'Maize and products', 'Milk - Excluding Butter', 
                      'Soyabeans', 'Sugar (Raw Equivalent)', 'Palm Oil', 'Vegetables, Other', 
                      'Barley and products', 'Rice (Milled Equivalent)', 'Oranges, Mandarines', 
                      'Cassava and products', 'Fruits, Other', 'Potatoes and products', 'Pelagic Fish', 
                      'Tomatoes and products' ])]

# Tri par ordre décroissant sur 'Import Quantity'
imp2.sort_values(['Import Quantity'], ascending=[False], inplace=True)
imp2.head(200)

,ctry_cd,country,item_cd,item,origin,population,Domestic supply quantity,Feed,Food,Import Quantity,Other uses
2509,41,"China, mainland",2555,Soyabeans,vegetal,1385567000,75081.0,9530.0,5072.0,63381.0,NaN
2496,41,"China, mainland",2532,Cassava and products,vegetal,1385567000,33590.0,22868.0,2645.0,29046.0,7940.0
6965,110,Japan,2514,Maize and products,vegetal,127144000,14661.0,10964.0,1171.0,14403.0,288.0
12012,194,Saudi Arabia,2513,Barley and products,vegetal,28829000,10317.0,10000.0,0.0,10547.0,NaN
3674,59,Egypt,2511,Wheat and products,vegetal,82056000,19341.0,4860.0,12048.0,10331.0,0.0
7490,117,Republic of Korea,2514,Maize and products,vegetal,49263000,8749.0,6281.0,614.0,8755.0,47.0
6069,100,India,2577,Palm Oil,vegetal,1252140000,8413.0,NaN,1631.0,8568.0,6782.0
4769,79,Germany,2848,Milk - Excluding Butter,animal,82727000,23999.0,2064.0,21401.0,8332.0,504.0
2581,41,"China, mainland",2848,Milk - Excluding Butter,animal,1385567000,48271.0,1608.0,45252.0,8207.0,NaN
6681,106,Italy,2848,Milk - Excluding Butter,animal,60990000,15918.0,524.0,15057.0,7695.0,301.0


In [151]:
# sélection des valeurs >=1092.0
impval=imp2.copy()
impval=impval[(impval['Import Quantity'] >= 1092.0)]

# vérification du nombre de lignes
impval['Import Quantity'].sum()

665953.0

### 11.4 - Groupez ces importations par produit, afin d'avoir une table contenant 1 ligne pour chacun des 15 produits. 

In [152]:
# Calcul de la somme de 'Export Quantity' pour chaque élément de 'item'
ratios = impval.groupby(impval['item']).sum()
ratios

,ctry_cd,item_cd,population,Domestic supply quantity,Feed,Food,Import Quantity,Other uses
item,,,,,,,,
Barley and products,850,17591,1852492000,29473.0,20153.0,338.0,22387.0,12.0
Cassava and products,689,12660,1775037000,67326.0,25149.0,15257.0,34655.0,20355.0
"Fruits, Other",866,18375,2094036000,72851.0,0.0,66962.0,15714.0,0.0
Maize and products,3836,67878,3161385000,687785.0,392162.0,55251.0,101416.0,181868.0
Milk - Excluding Butter,3714,74048,3150980000,341118.0,27211.0,298843.0,87377.0,12337.0
"Oranges, Mandarines",1230,20888,736332000,19534.0,0.0,19192.0,22945.0,1.0
Palm Oil,1222,25770,3533616000,25618.0,0.0,6833.0,29747.0,18568.0
Pelagic Fish,767,16578,1841107000,16094.0,12156.0,3937.0,13445.0,0.0
Potatoes and products,1321,20248,666233000,52519.0,1974.0,39435.0,16895.0,3427.0


#### 11.4.1 - calculez pour chaque produit le ratio entre la quantité destinés aux "Autres utilisations" (Other uses) et la disponibilité intérieure

In [153]:
# création d'une colonne ratio ou/dsq
ratio1 = ratios.copy()

# nouvelle colonne FS prot ctry item (g/day)
ratio1['ratio ou/dsq'] = ratio1['Other uses'] / ratio1['Domestic supply quantity']
# Tri par ordre décroissant sur 'ratio ou/dsq'
ratio1.sort_values(['ratio ou/dsq'], ascending=[False], inplace=True)
ratio1

,ctry_cd,item_cd,population,Domestic supply quantity,Feed,Food,Import Quantity,Other uses,ratio ou/dsq
item,,,,,,,,,
Palm Oil,1222,25770,3533616000,25618.0,0.0,6833.0,29747.0,18568.0,0.724803
Cassava and products,689,12660,1775037000,67326.0,25149.0,15257.0,34655.0,20355.0,0.302335
Maize and products,3836,67878,3161385000,687785.0,392162.0,55251.0,101416.0,181868.0,0.264426
Potatoes and products,1321,20248,666233000,52519.0,1974.0,39435.0,16895.0,3427.0,0.065253
Sugar (Raw Equivalent),2000,40672,4027824000,78453.0,0.0,64971.0,34208.0,2996.0,0.038188
Milk - Excluding Butter,3714,74048,3150980000,341118.0,27211.0,298843.0,87377.0,12337.0,0.036166
Wheat and products,6036,105462,4291411000,447942.0,95503.0,296612.0,146539.0,14397.0,0.032140
Rice (Milled Equivalent),1221,22440,1775478000,147352.0,12592.0,120417.0,12911.0,2533.0,0.017190
Soyabeans,2173,38325,2851790000,159570.0,13710.0,7510.0,93455.0,524.0,0.003284


#### 11.4.2 - calculez pour chaque produit le ratio entre la quantité destinée à la nourriture animale et la quantité destinée à la nourriture (animale + humaine)

In [154]:
# création d'une colonne ratio ani/food
ratio2 = ratios.copy()
ratio2['ratio ani/food'] = ratio2['Feed'] / (ratio2['Feed'] + ratio2['Food'])
# Tri par ordre décroissant sur 'ratio ani/food'
ratio2.sort_values(['ratio ani/food'], ascending=[False], inplace=True)
ratio2

,ctry_cd,item_cd,population,Domestic supply quantity,Feed,Food,Import Quantity,Other uses,ratio ani/food
item,,,,,,,,,
Barley and products,850,17591,1852492000,29473.0,20153.0,338.0,22387.0,12.0,0.983505
Maize and products,3836,67878,3161385000,687785.0,392162.0,55251.0,101416.0,181868.0,0.876510
Pelagic Fish,767,16578,1841107000,16094.0,12156.0,3937.0,13445.0,0.0,0.755359
Soyabeans,2173,38325,2851790000,159570.0,13710.0,7510.0,93455.0,524.0,0.646089
Cassava and products,689,12660,1775037000,67326.0,25149.0,15257.0,34655.0,20355.0,0.622408
Wheat and products,6036,105462,4291411000,447942.0,95503.0,296612.0,146539.0,14397.0,0.243559
Rice (Milled Equivalent),1221,22440,1775478000,147352.0,12592.0,120417.0,12911.0,2533.0,0.094670
Milk - Excluding Butter,3714,74048,3150980000,341118.0,27211.0,298843.0,87377.0,12337.0,0.083456
Potatoes and products,1321,20248,666233000,52519.0,1974.0,39435.0,16895.0,3427.0,0.047671


# Question 12 #
Donnez les 3 produits qui ont la plus grande valeur pour chacun des 2 ratios (vous aurez donc 6 produits à citer)

**Pour le ratio entre la quantité destinés aux "Autres utilisations" (Other uses) et la disponibilité intérieure-(ex 11.4.1)**
* Palm Oil (0.724803)
* Cassava and products (0.302335)
* Maize and products (0.264426)

**Pour le ratio entre la quantité destinée à la nourriture animale et la quantité destinée à la nourriture (animale + humaine)-(ex 11.4.1)**
* Barley and products (0.983505)
* Maize and products (0.876510)
* Pelagic Fish (0.755359)

# Question 13 #
Combien de tonnes de céréales pourraient être libérées si les USA diminuaient leur production de produits animaux de 10% ?

In [155]:
# on reprend le df 'ic' de l'ex 10.3
anusa = ic.copy()
anusa.head(0)

,ctry_cd,country,item_cd,item,origin,population,Domestic supply quantity,Export Quantity,Fat supply quantity (g/capita/day),Feed,...,Food supply quantity (kg/capita/yr),Import Quantity,Losses,Other uses,Processing,Production,Protein supply quantity (g/capita/day),Seed,Stock Variation,is_cereal


In [156]:
# on retire les colonnes inutiles
anusa.drop(['Production', 'Import Quantity', 'Stock Variation', 'Food',
       'Seed', 'Food supply (kcal/capita/day)','Domestic supply quantity',
       'Fat supply quantity (g/capita/day)', 'Export Quantity',
        'Processing', 'Other uses', 'Protein supply quantity (g/capita/day)','Food supply quantity (kg/capita/yr)','Losses','is_cereal'], axis=1, inplace = True)

# on sélectionne USA
anusa = anusa[anusa['ctry_cd'] == 231]
anusa

,ctry_cd,country,item_cd,item,origin,population,Feed
14263,231,United States of America,2511,Wheat and products,vegetal,320051000,6196.0
14264,231,United States of America,2513,Barley and products,vegetal,320051000,1440.0
14265,231,United States of America,2514,Maize and products,vegetal,320051000,128024.0
14266,231,United States of America,2515,Rye and products,vegetal,320051000,184.0
14267,231,United States of America,2516,Oats,vegetal,320051000,1430.0
14268,231,United States of America,2517,Millet and products,vegetal,320051000,394.0
14269,231,United States of America,2518,Sorghum and products,vegetal,320051000,2351.0
14270,231,United States of America,2520,"Cereals, Other",vegetal,320051000,77.0
14355,231,United States of America,2805,Rice (Milled Equivalent),vegetal,320051000,NaN


In [157]:
resusa = anusa.copy()

# on retire les NaN
resusa = resusa[resusa['Feed'].notnull()]

# somme de 'feed' divisée 0,1 (10%) multiplié par 100 (unit = 1000 tonnes)
resusa = resusa['Feed'].sum() * (0.1 * 1000)
print(resusa)

14009600.0


* En réduisant de 10% la consommation de céréales destinée à l'alimentation animale, les USA libèreraient 1.400.960 tonnes de céréales

# Question 14

### 14.1 - En Thaïlande, quelle proportion de manioc est exportée ? 

#### 14.1.1 Data wrangling
**Rappel de l'exercice 2**
* A = [Domestic supply quantity]
* B = [Export Quantity]
* C = [Import Quantity + Production + Stock Variation]
* C = A + B<br>

**La proportion (%) de manioc exportée =** [Export Quantity] / [Export Quantity]  + [Domestic supply quantity] * 100<br><br>

In [158]:
# on reprend 'data' du TP3
th2 = data.copy()
# sélection de 'Thaïlande' (ctry_cd = 216)
th2 = th2[(th2['ctry_cd'] == 216) & (th2['item'] == 'Cassava and products')] 
th2

,ctry_cd,country,item_cd,item,origin,population,Domestic supply quantity,Export Quantity,Fat supply quantity (g/capita/day),Feed,...,Food supply (kcal/capita/day),Food supply quantity (kg/capita/yr),Import Quantity,Losses,Other uses,Processing,Production,Protein supply quantity (g/capita/day),Seed,Stock Variation
13344,216,Thailand,2532,Cassava and products,vegetal,67011000,6264.0,25214.0,0.05,1800.0,...,40.0,13.0,1250.0,1511.0,2081.0,0.0,30228.0,0.14,NaN,0.0


In [159]:
# on retire les colonnes inutiles
th2.drop(['Seed', 'Food supply (kcal/capita/day)',
          'Fat supply quantity (g/capita/day)','Processing','Feed','Food','Other uses',
          'Protein supply quantity (g/capita/day)','Food supply quantity (kg/capita/yr)','Losses',
          'Import Quantity','Production','Stock Variation'], axis=1, inplace = True)
th2

,ctry_cd,country,item_cd,item,origin,population,Domestic supply quantity,Export Quantity
13344,216,Thailand,2532,Cassava and products,vegetal,67011000,6264.0,25214.0


In [160]:
# Calcul de la proportion de manioc exportée 'manioc expt (%)'
th2['manioc expt (%)'] = th2['Export Quantity'] / (th2['Export Quantity'] + th2['Domestic supply quantity']) * 100
th2

,ctry_cd,country,item_cd,item,origin,population,Domestic supply quantity,Export Quantity,manioc expt (%)
13344,216,Thailand,2532,Cassava and products,vegetal,67011000,6264.0,25214.0,80.100388


* plus de 80% de la disponibilité intérieure de manioc est exportée

### 14.2 - Quelle est la proportion de personnes en sous-nutrition?

In [161]:
# récupération du df 'snba' de la question 11.2
snth=snba.copy()
# sélection de 'Thaïlande' (ctry_cd = 216)
snth = snth[(snth['ctry_cd'] == 216)]
snth

,ctry_cd,undernourished
179,216,6100000.0


In [162]:
# récupération du df 'pop_clean' de la question 1
popth=pop_clean.copy()
# sélection de 'Thaïlande' (ctry_cd = 216)
popth = popth[(popth['ctry_cd'] == 216)]
popth

,ctry_cd,population
152,216,67011000


In [163]:
# calcul de la proportion (%)
6100000 / 67011000 * 100

9.102983092328126

* 9,1% de la population estt en sous-nutrition selon les critères de la FAO